In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import os
from datetime import datetime, UTC
import analytiq_data as ad
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
import docrouter_app.payments

In [4]:
ad.common.setup()
await docrouter_app.payments.init_payments_env()

In [5]:
#await docrouter_app.payments.delete_all_payments_customers(dryrun=False)

In [6]:
org_id = "685c1fb8e8952690f233e049"

In [7]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

2025-06-30 12:39:09,898 - docrouter_app.payments - INFO - Sync stripe customer for org_id: 685c1fb8e8952690f233e049 user_id: 685c1fb8e8952690f233e048 email: andrei@analytiqhub.com name: andrei@analytiqhub.com org_name: andrei@analytiqhub.com
2025-06-30 12:39:09,903 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27
2025-06-30 12:39:10,148 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27 response_code=200
2025-06-30 12:39:10,151 - docrouter_app.payments - INFO - No Stripe customers found for org_id: 685c1fb8e8952690f233e049
2025-06-30 12:39:10,152 - stripe - INFO - message='Request to Stripe api' method=post url=https://api.stripe.com/v1/customers
2025-06-30 12:39:10,661 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers resp

{'org_id': '685c1fb8e8952690f233e049',
 'user_id': '685c1fb8e8952690f233e048',
 'stripe_customer_id': 'cus_SawcfsptWMvKKf',
 'user_name': 'andrei@analytiqhub.com',
 'user_email': 'andrei@analytiqhub.com',
 'has_payment_method': False,
 'created_at': datetime.datetime(2025, 6, 30, 16, 39, 12, 201949),
 'updated_at': datetime.datetime(2025, 6, 30, 16, 39, 12, 201957),
 '_id': ObjectId('6862bdb09aa7101582f60496')}

In [ ]:
stripe_customer = await docrouter_app.payments.get_payments_customer(org_id=org_id)

In [ ]:
stripe_customer

In [ ]:
stripe_customer.id

In [ ]:
subscription = await docrouter_app.payments.get_subscription(stripe_customer.id)
if subscription:
    print(docrouter_app.payments.get_subscription_type(subscription))
else:
    print("No subscription found")

In [8]:
await docrouter_app.payments.get_stripe_usage(org_id=org_id)

2025-06-30 12:40:56,066 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27
2025-06-30 12:40:56,308 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27 response_code=200
2025-06-30 12:40:56,311 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf
2025-06-30 12:40:56,481 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf response_code=200
2025-06-30 12:40:56,483 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/billing/meters/mtr_test_61SCqwDks28Lo6CVN41Q21OOWmU21HJQ/event_summaries?customer=cus_SawcfsptWMvKKf&start_time=1751301550&end_time=1753893550
2025-06-30 12:40:56,664 - st

2025-06-30 12:40:56,482 - docrouter_app.payments - INFO - Current period start: 1751301550
2025-06-30 12:40:56,483 - docrouter_app.payments - INFO - Current period end: 1753893550
2025-06-30 12:40:56,666 - docrouter_app.payments - INFO - Usage summary: {
  "data": [],
  "has_more": false,
  "object": "list",
  "url": "/v1/billing/meters/mtr_test_61SCqwDks28Lo6CVN41Q21OOWmU21HJQ/event_summaries"
}


{'total_usage': 0,
 'included_usage': 500,
 'overage_usage': 0,
 'remaining_included': 500,
 'current_period_start': 1751301550,
 'current_period_end': 1753893550,
 'period_start': 1751301550,
 'period_end': 1753893550,
 'subscription_type': 'team',
 'usage_unit': 'spu'}

In [ ]:
await docrouter_app.payments.get_stripe_usage(org_id=org_id)

2025-06-30 12:40:56,066 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27
2025-06-30 12:40:56,308 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/customers/search?query=metadata[%27org_id%27]%3A%27685c1fb8e8952690f233e049%27 response_code=200
2025-06-30 12:40:56,311 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf
2025-06-30 12:40:56,481 - stripe - INFO - message='Stripe API response' path=https://api.stripe.com/v1/subscriptions?customer=cus_SawcfsptWMvKKf response_code=200
2025-06-30 12:40:56,483 - stripe - INFO - message='Request to Stripe api' method=get url=https://api.stripe.com/v1/billing/meters/mtr_test_61SCqwDks28Lo6CVN41Q21OOWmU21HJQ/event_summaries?customer=cus_SawcfsptWMvKKf&start_time=1751301550&end_time=1753893550
2025-06-30 12:40:56,664 - st

2025-06-30 12:40:56,482 - docrouter_app.payments - INFO - Current period start: 1751301550
2025-06-30 12:40:56,483 - docrouter_app.payments - INFO - Current period end: 1753893550
2025-06-30 12:40:56,666 - docrouter_app.payments - INFO - Usage summary: {
  "data": [],
  "has_more": false,
  "object": "list",
  "url": "/v1/billing/meters/mtr_test_61SCqwDks28Lo6CVN41Q21OOWmU21HJQ/event_summaries"
}


{'total_usage': 0,
 'included_usage': 500,
 'overage_usage': 0,
 'remaining_included': 500,
 'current_period_start': 1751301550,
 'current_period_end': 1753893550,
 'period_start': 1751301550,
 'period_end': 1753893550,
 'subscription_type': 'team',
 'usage_unit': 'spu'}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [ ]:
price_id = docrouter_app.payments.TIER_TO_PRICE["individual"]
price_id

In [16]:
# Use the new subroutine
subscription_type = docrouter_app.payments.get_subscription_type(price_id)

# Create the subscription
subscription = await docrouter_app.payments.StripeAsync.subscription_create(
    customer=stripe_customer.id,
    items=[{
        "price": price_id,
    }],
    payment_behavior="default_incomplete",
    expand=["latest_invoice.payment_intent"],
)

# Find the subscription item ID for usage reporting later
subscription_item_id = subscription["items"]["data"][0]["id"]

In [ ]:
subscription

In [ ]:
await docrouter_app.payments.record_usage(org_id=org_id, pages_processed=10, operation="test")




In [38]:
org_id = "6859696f6ebcee0b712b9ac6"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
await docrouter_app.payments.delete_payments_customer(org_id=org_id, force=False)